## Neville Modise - Linear Knapsack Problem

The Knapsack Problem is considered to be a combinatorial optimization problem. The best selection and configuration of a collection of objects adhering to some objective function defines
combinatorial optimization problems. A Knapsack model serves as an abstract model with broad
spectrum applications such as: Resource allocation problems, Portfolio optimization, Cargo-loading
problems


## Linear Knapsack problem.

We wish to maximize $\sum_{i=1}^{n}v_ix_i$ subject to $\sum_{i=1}^{n}w_ix_i\leq W$.<br> Where $v_i$ and $w_i$ represent the value and weight of the  $i^{th}$ item respectively and <br>$x_i\in\{1,0\}$ represents whether the item is taken or not and $W$ is the <br>maximum allowed weight for the set of items <br>

$(v_i, w_i)=${(2,7),(6,3),(8,3),(7,5),<br>(3,4),(4,7),(6,5),(5,4),
(10,15),<br>(9,10),(8,17),(11,3),(12,6),(15,11),<br>(6,6),(8,14),(13,4),
(14,8),(15,9),<br>(16,10),(13,14),(14,17),(15,9),(26,24),<br>
(13,11),(9,17),(25,12),(26,14)}<br> with a total capacity $W=30$

### Greedy Algorithm

A greedy approach can be implemented with the following steps
- Use a score/efficiency function i.e a profit to weight ratio that is $$f_i=\frac{v_i}{w_i}$$

$\quad$ if $f_i\geq f_j$ it means we prefer item $i$ over item $j$
- Sort the items according to their efficiency score (descending order)
- Continuous add items with the highest score to the knapsack without violating the weight limit
- Return the set of items that satisfies weight limit and yields maximum profit

In [1]:
import numpy as np
import random
# let's create an instance for the knapsack problem
value = np.array([2,6,8,7,3,4,6,5,10,9,8,11,12,15,6,8,13,14,15,16,13,14,15,26,13,9,25,26])
weight = np.array([7,3,3,5,4,7,5,4,15,10,17,3,6,11,6,14,4,8,9,10,14,17,9,24,11,17,12,14])
# define max weight for the knapsack
maxWeight=30
# number of elements in a solution
n=28


In [2]:
def knapsackGreProc(value,weight,W):
    
# sorting Item on basis of ratio
    score = value/weight #calculate Score
    
    scoreindex = np.argsort(-score)#sort in descending order
    weight=weight[scoreindex]
#add the weights without exceeding the weight capacity 30
    sumofitems = 0
    indx_binary=np.zeros(n).astype(int)
    for i in range(n):
# If adding Item won't overflow, add it completely
       if ((sumofitems+weight[i])<=W):
        indx_binary[scoreindex[i]]=1
        sumofitems=sumofitems+weight[i]
    return indx_binary

In [3]:
print("Question 1 - Greedy Algorithm Implementation ")
indx_binary =knapsackGreProc(value,weight,maxWeight)
print("Profit :",np.sum(value[indx_binary==1]))
print("Weight of the knapsack :",np.sum(weight[indx_binary==1]))
print("Selected value :",value[indx_binary ==1])
print("Selected weight :",weight[indx_binary ==1])


Question 1 - Greedy Algorithm Implementation 
Profit : 70
Weight of the knapsack : 30
Selected value : [ 6  8  7 11 13 25]
Selected weight : [ 3  3  5  3  4 12]


The maximum value we can obtain is 70 with the weight of the knapsack at 30

### Question 2 - Iterative Improvement Local Search

We present a modified local search method for solving binary integer programming problems. This
local search method will be referred to as the ‘iterative improvement local search’ (IILS). We want
to maximize the linear KP problem in Q1 via maximizing the penalty function using the iterative
improvement local search (IILS).


In [4]:
from random import Random
# number of elements in a solution
n = 28

# maximum iteration
iteration = 500

# maximum number of flip
MaxFlip = 2
flip = 1


# penalty function to evaluate a solution x
def evaluate(x):
    a = np.array(x)
    b = np.array(values)
    c = np.array(weights)
    

    value = np.dot(a, b)  # compute the cost value of the knapsack selection
    totalWeight = np.dot(a, c)  # compute the weight value of the knapsack selection

    if totalWeight > maxWeight:
        value = maxWeight - totalWeight
    return [value, totalWeight]  



# function to create a 1-flip neighborhood of solution x
def neighborhood(x):
    if flip == 1:
        nbrhood = []

        for i in range(0, n):
            nbrhood.append(x[:])
            if nbrhood[i][i] == 1:
                nbrhood[i][i] = 0
            else:
                nbrhood[i][i] = 1

    elif flip == 2:
        nbrhood = []
        a = -1
        for i in range(0, n):
            for j in range(i, n):
                if i != j:
                    a += 1
                    nbrhood.append(x[:])

                    if nbrhood[a][i] == 1:
                        nbrhood[a][i] = 0
                    else:
                        nbrhood[a][i] = 1
                    # end

                    if nbrhood[a][j] == 1:
                        nbrhood[a][j] = 0
                    else:
                        nbrhood[a][j] = 1

    else:
        nbrhood = []
        for i in range(0, n):
            nbrhood.append(x[:])
            nbrhood[i][i], nbrhood[i][i-3]=nbrhood[i][i-3], nbrhood[i][i]

    return nbrhood

# to setup a random number generator, we will specify a "seed" value
seed = 5113
myPRNG = Random(seed)

# let's create an instance for the knapsack problem
values = [2,6,8,7,3,4,6,5,10,9,8,11,12,15,6,8,13,14,15,16,13,14,15,26,13,9,25,26]



weights = [7,3,3,5,4,7,5,4,15,10,17,3,6,11,6,
                 14,4,8,9,10,14,17,9,24,11,17,12,14]



# define max weight for the knapsack
maxWeight = 30

# monitor the number of solutions evaluated
solutionsChecked = 0

# define the solution variables
# x_curr will hold the current solution
# f_curr will hold the "fitness" of the current soluton
# x_best will hold the best solution
x_curr = []

# start with a random solution
for i in range(0, n):
    if myPRNG.random() < 0.7:
        x_curr.append(0)
    else:
        x_curr.append(1)

# begin local search overall logic
x_best = x_curr[:]
f_curr = evaluate(x_curr)[0]
totalWeight = evaluate(x_curr)[1]
f_best = f_curr

List = {}

for ite in range(iteration):
    flip = 1
    while flip < MaxFlip + 1:

        # create a list of all neighbors in the neighborhood of x_curr
        Neighborhood = neighborhood(x_curr)

        # selecting best neighbor
        x_neigh = Neighborhood[0]
        f_neigh = evaluate(x_neigh)[0]
        totalWeight_neigh = evaluate(x_neigh)[1]

        for s in Neighborhood:
            solutionsChecked = solutionsChecked + 1

            if evaluate(s)[0] > f_neigh:
                if evaluate(s)[0] > f_best:
                        # find the best member and keep track of that solution
                        x_neigh = s[:]
                        f_neigh = evaluate(s)[0]
                        totalWeight_neigh = evaluate(s)[1]

        # update the current list
        x_curr_old = x_curr
        x_curr = x_neigh

        # check if it is maximum
        if f_neigh > f_best:
            x_best = x_neigh
            f_best = f_neigh           #value
            totalWeight = totalWeight_neigh    #weight

            flip = MaxFlip + 1
        else:
            flip += 1


    #
    old_list = List
    List = {}

print("\nFinal: Total number of solutions checked: ", solutionsChecked)
print("Best value found: ", f_best)
print("Weight of knapsack: ", totalWeight)
print("Maximum weight of the knapsack: ", maxWeight)
print("Best solution: ", x_best)



Final: Total number of solutions checked:  200354
Best value found:  66
Weight of knapsack:  30
Maximum weight of the knapsack:  30
Best solution:  [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
